<h1><center>CS457 Lab 2 <br />UDP Pinger</center></h1>

### Overview

You had a brief and simplified introduction to **TCP** in lab 1.  There is still *much* to learn about TCP, and we will do that in future labs.  But for now let's get a good understanding of the **UDP** protocol.

In this lab, you will learn the basics of socket programming for **UDP** in Python. You will learn how to send and receive datagram packets using UDP sockets and also, how to set a proper socket timeout. Throughout the lab, you will gain familiarity with a Ping application and its usefulness in computing statistics such as packet loss rate.
You will first study a simple Internet ping server written in the Python, and implement a corresponding client. The functionality provided by these programs is similar to the functionality provided by standard ping programs available in modern operating systems. However, these programs use a simpler protocol, UDP, rather than the standard Internet Control Message Protocol (ICMP) to communicate with each other. The ping protocol allows a client machine to send a packet of data to a remote machine, and have the remote machine return the data back to the client unchanged (an action referred to as echoing). Among other uses, the ping protocol allows hosts to determine round-trip times to other machines.

Unlike lab1, this lab requires you to compose new python code for the UDPPinger client.  A skeleton framework is given, you will need to fill in the blanks. 

#### Acknowledgements and Citations

This lab has original material as well as material sourced and modified from the Kurose and Ross Networking textbook's set of lab exercises.


### UDP socket programming: brief tutorial

UDP is a much simpler protocol than TCP.  It is a "message-based" protocol, not a streaming protocol.  In UDP, there is no need to establish a connection with a three-way handshake.  There is no state maintained.  There is no need to listen or accept a connection.  UDP is basically a simple way to send or receive a complete message (datagram). The most common example of a UDP-based application is DNS.  Streaming video and audio also use the UDP protocol.

You can see this simplicity by comparing the UDP flow diagram below to the one for TCP presented in lab 1.

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/UDPsockets.jpg" alt="UDP flow diagram" />

Notice the two new socket calls:

* `sendto()` instead of `send()` 
* `recvfrom()` instead of `recv()`

These two functions take two parameters, the message and an address.  The address parameter is needed because, as stated earlier, there is no state kept in the socket object.  The address parameter is actually a tuple, `(IPv4Addr, port).`  

Unlike TCP, UDP provides applications with an *unreliable transport service* due to the "best effort" nature of sending datagrams. Messages may get lost in the network due to heavy congestion that create router queue overflows, faulty hardware or other reasons.  Typically this is not an issue, and most request messages are sent and received ok.  But if a UDP request or response (which are based on the UDP protocol) does gets lost the user or application can simply re-issue the DNS request.  Applications that use DNS have to accomodate this behavior. 

---
### Server Code
The following code fully implements the ping server. In this server code, approximately 40% of the client’s packets are simulated to be lost. You should study this code carefully, as it will help you write your ping client.  *Note: You do not need to modify the server code.*

You should execute the jupyter cell below to save the server code to your current working directory.  Then use a terminal window to navigate to that directory and start the server using the command:
>python cs457_UDPPingerServer.py

The program will sit and wait for messages from the client. Before we get to that step, let's analyze the server code. 


In [1]:
%%writefile cs457_UDPPingerServer.py

import random
from socket import *

# Create a UDP socket
# Notice the use of SOCK_DGRAM for UDP packets 

serverSocket = socket(AF_INET, SOCK_DGRAM)

# Assign IP address and port number to socket 

serverSocket.bind(('127.0.0.1', 12000))

while True:
    # Generate random number in the range of 0 to 10
    rand = random.randint(0, 10)
    # Receive the client packet along with the address it is coming from 
    message, address = serverSocket.recvfrom(1024)
    # Capitalize the message from the client
    message = message.upper()
    # If rand is less is than 4, we consider the packet lost and do not respond 
    if rand < 4:
        continue
    # Otherwise, the server responds 
    serverSocket.sendto(message, address)

Writing cs457_UDPPingerServer.py


#### Server main loop

The server sits in an infinite loop listening for incoming UDP packets. When a packet comes in and if a randomized integer is greater than or equal to 4, the server simply capitalizes the encapsulated data and sends it back to the client.

#### Packet Loss
UDP provides applications with an *unreliable transport service*. Messages may get lost in the network due to router queue overflows, faulty hardware or some other reasons. Because packet loss is rare or even non-existent in typical campus networks, the server in this lab injects artificial loss to simulate the effects of network packet loss. The server creates a variable randomized integer which determines whether a particular incoming packet is lost or not.

### Client Code
You need to implement the client program yourself.  A skeleton program is given below as an aid.

The client should send 10 pings to the server. Because UDP is an unreliable protocol, a packet sent from the client to the server may be lost in the network, or vice versa. For this reason, the client cannot wait indefinitely for a reply to a ping message. You should get the client wait up to one second for a reply; if no reply is received within one second, your client program should assume that the packet was lost during transmission across the network. You will need to look up the [python socket documentation](https://docs.python.org/3/library/socket.html) to find out how to set the timeout value on a datagram socket.  There is a lot of other useful information here as well, but most of it is irrelevant to this particular lab exercise.

Specifically, your client program should

1. send the ping message using UDP (Note: Unlike TCP, you do not need to establish a connection first, since UDP is a connectionless protocol.

2. print the response message from server, if any

3. calculate and print the round trip time (RTT), in seconds, of each packet, if server responses

4. otherwise, print “Request timed out”

During development, you should run the UDPPingerServer.py from a terminal window, as mentioned earlier.

Test your client by sending packets to localhost (or, 127.0.0.1). After you have fully debugged your code, you should see how your application communicates across the network with the ping server and ping client running on different machines.

#### Message Format

The ping messages in this lab are formatted in a simple way. The client message is one line, consisting of ASCII characters in the following format:
>`Ping sequence_number time`

where `sequence_number` starts at 1 and progresses to 10 for each successive ping message sent by the client, and `time` is the time when the client sends the message.  The server will convert the message to upper-case and return it to you in its echo response.

This program should take less than 20 lines of python code.  But don't make that a goal.  A working client should be your goal.  

The  lines with `YOU FILL IN` is where you write code.  This may seem like I'm spoon-feeding you, but that's OK since this is your first UDP program.  Later on I won't be as nice. You can count on that!

The output from your client should look something like this:

```
Request timed out.
Request timed out.
Reply from 127.0.0.1: PING 3 FRI JUL 19 18:27:46 2019
RTT: 0.0003991127014160156
Reply from 127.0.0.1: PING 4 FRI JUL 19 18:27:46 2019
RTT: 0.0002460479736328125
Reply from 127.0.0.1: PING 5 FRI JUL 19 18:27:46 2019
RTT: 0.00021982192993164062
Request timed out.
Reply from 127.0.0.1: PING 7 FRI JUL 19 18:27:47 2019
RTT: 0.0003857612609863281
Reply from 127.0.0.1: PING 8 FRI JUL 19 18:27:47 2019
RTT: 0.00022602081298828125
Reply from 127.0.0.1: PING 9 FRI JUL 19 18:27:47 2019
RTT: 0.0002570152282714844
Reply from 127.0.0.1: PING 10 FRI JUL 19 18:27:47 2019
RTT: 0.00020599365234375
```

In [38]:
import sys, time, datetime
from socket import *

# Initialization Section
#    Set host, port and timeout
host = '129.82.44.158' #'127.0.0.1'
port = 12004
timeout = 1.0 # in second
#    YOU FILL IN: Create UDP client socket, set its timeout, and initialize a variable for the PING sequence number

# PING sequence count
ping_sequence = 1
ping_message = "ping"

# socket
clientsocket = socket(AF_INET, SOCK_DGRAM)
# bind
#clientsocket.bind((host, port))


#################### EXTRA CREDIT
successes = 0
total_time = 0.0
minimum_time = 100.0
maximum_time = 0.0
####################

# Ping for 10 times using a while loop
while ping_sequence <= 10:
    

    #increment the sequence number
    # YOU FILL IN: Format a string message to be sent per the instructions above (hint: time.asctime() is nice)
    
    # Put the rest in a try/except block to handle any possible timeouts
    
    try:
        # create datetime object for stamp
        date_for_ping = datetime.datetime.now().strftime("%c") # didn't read the 'time.asctime()' thing in time, oh well
        ping_message = "ping " + str(ping_sequence) + " " + date_for_ping
        
        # actually send encoded message
        clientsocket.sendto(ping_message.encode('utf_8'),("129.82.44.158", port))
        # save send time
        ping_time_start = time.time()
        # set timeout?
        clientsocket.settimeout(timeout)
        
        # get the response
        response_message , response_addy = clientsocket.recvfrom(1024)

        print("Reply from {0}: {1}".format(response_addy[0], response_message.decode()))
        ping_time_end = time.time() - ping_time_start
        print("RTT: {0}".format(ping_time_end))
        
        #################### EXTRA CREDIT
        total_time += ping_time_end
        successes += 1
        
        # check mins and maxs
        if minimum_time > ping_time_end:
            minimum_time = ping_time_end
            
        if maximum_time < ping_time_end:
            maximum_time = ping_time_end
        #################### 
        
        
        
    except Exception as error:
        # print(str(error))
        # Server does not respond.  Assume the packet is lost
        # Assume the packet is lost
        print ("Request timed out.")
        continue
    finally:
        # Increment count
        ping_sequence += 1
        

# Close the client socket
clientsocket.close()

#################### EXTRA CREDIT
print("\n")
print("Total packets transmitted: {0}".format(10))
print("Total packets recieved:    {0}".format(successes))
print("Success rate:              {0}%".format(successes*10))
print("Maximum packet time:       {0}".format(maximum_time))
print("Minimum packet time:       {0}".format(minimum_time))
print("Average packet time:       {0}".format(total_time/successes))
#print("{0:8.2f}")

####################



Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.
Request timed out.


Total packets transmitted: 10
Total packets recieved:    0
Success rate:              0%
Maximum packet time:       0.0
Minimum packet time:       100.0


ZeroDivisionError: float division by zero

### What to Hand in

Take a screen shot (or copy/paste) your complete client code and its output verifying that your ping program works on localhost as required.  Repeat the exercise running the server on a separate lab machine.  Take a screen shot of the new output from your client.  Answer the following question:

>Approximately how much longer does it take to do a round-trip ping from/to a lab machine than from/to localhost?  (Note,  answers may vary if you are doing the experiment from your home or from the CS building itself).

Submit this as a text or file to CANVAS.

#### Optional Exercises for extra credit

This lab is worth 100 points, as usual.  If you do the following, you will get 25 extra credit points on this exercise.

>Currently, the program calculates the round-trip time for each packet and prints it out individually. Modify this to correspond to the way the standard ping program works. You will need to report the minimum, maximum, and average RTTs at the end of all pings from the client. In addition, calculate the packet loss rate (in percentage).  Doing this will give you 